In [59]:
import pandas as pd
import numpy as np

In [60]:
# --- Load ---
flights = pd.read_csv(r"D:\Project DS Final\Data\clean_data_delay_Flights.csv")
weather = pd.read_csv(r"D:\Project DS Final\Data\wea_da_clean.csv")

# --- Xử lý thời gian cho flights ---
flights["DEP_HOUR"] = np.floor(flights["CRS_DEP_TIME"]).astype(int)

# Nếu có YEAR, dùng trực tiếp; nếu không, tạm đặt 2025
flights["FL_DATE"] = pd.to_datetime(
    dict(year=2025, month=flights["MONTH"], day=flights["DAY_OF_MONTH"])
)

# --- Chuẩn hóa cột thời tiết ---
if weather["date"].str.len().min() <= 5:  # ví dụ '04-01'
    weather["date"] = "2025-" + weather["date"]  # thêm năm tạm
weather["date"] = pd.to_datetime(weather["date"], errors="coerce")

# --- Merge ---
merged = flights.merge(
    weather,
    left_on=["ORIGIN", "FL_DATE", "DEP_HOUR"],
    right_on=["STATION", "date", "hour"],
    how="left"
)

# --- 4️⃣ Hiển thị thông tin kiểm tra ---
print("Kích thước dữ liệu sau merge:", merged.shape)
print("Số lượng giá trị thiếu theo cột:")
print(merged.isna().sum())


Kích thước dữ liệu sau merge: (50186, 27)
Số lượng giá trị thiếu theo cột:
MONTH                         0
DAY_OF_MONTH                  0
DAY_OF_WEEK                   0
OP_UNIQUE_CARRIER             0
ORIGIN                        0
ORIGIN_STATE_ABR              0
DEST                          0
DEST_STATE_ABR                0
CRS_DEP_TIME                  0
DEP_DELAY                     0
DEP_DELAY_NEW                 0
DEP_DEL15                     0
DEP_DELAY_GROUP               0
CRS_ARR_TIME                  0
CRS_ELAPSED_TIME              0
DISTANCE                      0
DISTANCE_GROUP                0
DEP_HOUR                      0
FL_DATE                       0
STATION                      94
date                         94
hour                         94
HourlyDewPointTemperature    94
HourlyDryBulbTemperature     94
HourlyRelativeHumidity       94
HourlyVisibility             94
HourlyWindSpeed              94
dtype: int64


In [61]:
# --- 5️⃣ Xem 10 dòng đầu để kiểm tra kết quả merge ---
print("\nCác dòng đầu tiên sau merge:")
print(merged.head(10))


Các dòng đầu tiên sau merge:
   MONTH  DAY_OF_MONTH  DAY_OF_WEEK OP_UNIQUE_CARRIER ORIGIN ORIGIN_STATE_ABR  \
0      4             1            2                AA    GEG               WA   
1      4             1            2                AA    GEG               WA   
2      4             1            2                AA    GEG               WA   
3      4             1            2                AA    SEA               WA   
4      4             1            2                AA    SEA               WA   
5      4             1            2                AA    SEA               WA   
6      4             1            2                AA    SEA               WA   
7      4             1            2                AA    SEA               WA   
8      4             1            2                AA    SEA               WA   
9      4             1            2                AA    SEA               WA   

  DEST DEST_STATE_ABR  CRS_DEP_TIME  DEP_DELAY  ...  DEP_HOUR    FL_DATE  \
0 

In [62]:
cols_to_drop = ["DEP_HOUR", "FL_DATE", "STATION", "date", "hour"]
merged.drop(columns=cols_to_drop, inplace=True, errors="ignore")

print("Kích thước sau khi xóa:", merged.shape)
print("Các cột hiện còn:", list(merged.columns))


Kích thước sau khi xóa: (50186, 22)
Các cột hiện còn: ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_ABR', 'DEST', 'DEST_STATE_ABR', 'CRS_DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE', 'DISTANCE_GROUP', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyRelativeHumidity', 'HourlyVisibility', 'HourlyWindSpeed']


In [63]:
# Danh sách các cột thời tiết
weather_cols = [
    "HourlyDewPointTemperature",
    "HourlyDryBulbTemperature",
    "HourlyRelativeHumidity",
    "HourlyVisibility",
    "HourlyWindSpeed"
]

# Các ký hiệu lỗi cần loại bỏ
invalid_values = ["*", "", " ", "M", "T", "0.50s", "10.00s", "9999", "-9999"]

# 1️⃣ Thay thế giá trị lỗi bằng NaN
merged[weather_cols] = merged[weather_cols].replace(invalid_values, np.nan)

# 2️⃣ Ép kiểu về float (có thể tạo thêm NaN nếu giá trị không hợp lệ)
merged[weather_cols] = merged[weather_cols].apply(pd.to_numeric, errors="coerce")

# 3️⃣ Drop các hàng có NaN trong cột thời tiết
before = merged.shape[0]
merged.dropna(subset=weather_cols, inplace=True)
after = merged.shape[0]

print(f"🧹 Đã xóa {before - after:,} dòng có dữ liệu thời tiết lỗi hoặc thiếu.")
print(f"📊 Dữ liệu còn lại: {after:,} dòng.")


🧹 Đã xóa 157 dòng có dữ liệu thời tiết lỗi hoặc thiếu.
📊 Dữ liệu còn lại: 50,029 dòng.


In [64]:
# --- Lọc các dòng có DEP_DELAY < 35 ---
before_filter = merged.shape[0]
merged = merged[merged["DEP_DELAY"] <= 35]
after_filter = merged.shape[0]

In [65]:
merged.to_csv(r"D:\Project DS Final\Data\final_clean_data.csv", index=False)